In [1]:
import ufl

import dolfinx
from dolfinx import mesh, fem, io, nls
from dolfinx.fem import FunctionSpace, VectorFunctionSpace
from dolfinx.fem import locate_dofs_geometrical
from dolfinx.fem.petsc import LinearProblem

from mpi4py import MPI
import numpy as np
import matplotlib.pyplot as plt

import shutil
import typing


# Operators:
class Infix:

    def __init__(self, function):
        self.function = function

    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))

    def __or__(self, other):
        return self.function(other)

    def __call__(self, value1, value2):
        return self.function(value1, value2)


dot = Infix(ufl.dot)
inner = Infix(ufl.inner)


def vector(*args):
    return ufl.as_vector(tuple(args))


from ufl import TrialFunction, TestFunction, FacetNormal, SpatialCoordinate
from ufl import dx, ds, exp, sym, tr, sqrt
from ufl import nabla_div as div
from ufl import nabla_grad as grad
from ufl import as_matrix as matrix
from ufl import lhs, rhs

npor = Infix(np.logical_or)
npand = Infix(np.logical_and)


def I(func_like):
    """Create matrix Identity dimension of func_like

    Args:
        func_like (Function): Give geometric dimension

    Returns:
        Tensor: Identity
    """
    return ufl.Identity(func_like.geometric_dimension())


# Classes:


def Function(space, func=None):
    """Function on new space. Without func just create a Function on space. 
    Args:
        space (FunctionSpace): New space
        function (): 
    \nScalars - fem.Function,fem.Constant, ufl_function, callable function, number
    \nVectors - fem.vector_Function,fem.vector_Constant, ufl_vector_function, callable vector_function, tuple_number
    Returns:
        fem.Function: Function
    """

    result = fem.Function(space)
    tupe = str(func.__class__)[8:-2]
    cord = SpatialCoordinate(space)

    # None
    if func == None:
        return result

    # fem.Function
    elif tupe == ('dolfinx.fem.function.Function'):
        expression = func

    # fem.Constant
    elif tupe == ('dolfinx.fem.function.Constant'):
        if len(func.ufl_shape) == 0: func2 = func.value + (cord[0] - cord[0])
        else:
            func2 = vector(*func.value) +\
                vector(*map(lambda x, y: x - y, cord, cord))
        expression = fem.Expression(func2, space.element.interpolation_points())

    # ufl object
    elif tupe[:3] == 'ufl':
        if len(func.ufl_shape) != 0:
            func2 = func + vector(*map(lambda x, y: x - y, cord, cord))
        else:
            func2 = func
        expression = fem.Expression(func2, space.element.interpolation_points())

    # Python function
    elif tupe == ('function'):
        expression = func

        # Number
    elif not hasattr(func, '__call__'):
        if hasattr(func, '__getitem__'):
            func2 = vector(*func) + vector(*map(lambda x, y: x - y, cord, cord))
        else:
            func2 = func + (cord[0] - cord[0])
        expression = fem.Expression(func2, space.element.interpolation_points())

    result.interpolate(expression)
    return result


def Constant(domain_space, const):
    """Constant on space

    Args:
        space (fem.FunctionSpace| domain): Space or domain
        const (auny number): Any number

    Returns:
        fem.function.Constant: Constant on space
    """
    return fem.Constant(domain_space, petsc4py.PETSc.ScalarType(const))


def create_facet_markers(domain, bound_markers):
    """Mark facets under conditious

    Args:
        domain (Domain): Domain
        bound_markers (mark,python_function): List of mark and function
    
    Return:
        tags(mesh.meshtags): Marked facets
    """
    facet_indices, facet_markers = [], []
    for (marker, condition) in bound_markers:
        facets = mesh.locate_entities(
            domain,
            domain.topology.dim - 1,
            condition,
            )
        facet_indices.append(facets)
        facet_markers.append(np.full_like(facets, marker))
    facet_indices = np.hstack(facet_indices).astype(np.int32)
    facet_markers = np.hstack(facet_markers).astype(np.int32)
    sorted_facets = np.argsort(facet_indices)
    facet_tags = mesh.meshtags(
        domain,
        domain.topology.dim - 1,
        facet_indices[sorted_facets],
        facet_markers[sorted_facets],
        )

    return facet_tags


def create_connectivity(domain):
    """Need to compute facets to Boundary value

    Args:
        domain (Mesh): Domain
    """
    domain.topology.create_connectivity(
        domain.topology.dim - 1,
        domain.topology.dim,
        )


def DirichletBC(space, func, combined_marker):
    """Create Dirichlet condition

    Args:
        space (fem.FunctionSpace): Function space
        func (fem.function): Function or Constant
        combined_marker (Any): One from next
        \nFunction - boundary marker function
        \nAll - all boundary
        \n(mesh.meshtags, marker) - list or tuple, marker of boundary from Marked_facets - mesh.meshtags
        
    Returns:
        condition (dirichletbc): Dirichlet condition
    """

    def get_exterior_facets():
        exterior_facets = mesh.exterior_facet_indices(domain.topology)
        return exterior_facets

    def all_dirichlet(dofs, func):
        if hasattr(func, 'function_space'):
            bc = fem.dirichletbc(dofs=dofs, value=func)
        else:
            bc = fem.dirichletbc(V=space, dofs=dofs, value=func)
        return bc

    domain = space.mesh

    if combined_marker == 'All':
        facets = get_exterior_facets()
        dofs = fem.locate_dofs_topological(
            space,
            domain.topology.dim - 1,
            facets,
            )

    elif type(combined_marker) == (tuple or list):
        marked_facets, marker = combined_marker
        facets = marked_facets.find(marker)
        dofs = fem.locate_dofs_topological(
            space,
            domain.topology.dim - 1,
            facets,
            )
    else:
        dofs = fem.locate_dofs_geometrical(space, marker=combined_marker)

    bc = all_dirichlet(dofs, func)

    return bc


# Solvers:
def NonLinear_Solver(problem, params={}, options={}, report=False):

    params0 = {
        'convergence_criterion': 'incremental',
        'tolerance': 1E-6,
        }
    params0.update(params)

    options0 = {
        'ksp_type': 'cg',
        'pc_type': 'gamg',
        'pc_factor_mat_solver_type': 'mumps',
        }
    options0.update(options)

    solver = nls.petsc.NewtonSolver(MPI.COMM_WORLD, problem=problem)

    solver.convergence_criterion = params0['convergence_criterion']
    solver.rtol = params0['tolerance']
    solver.report = report

    ksp = solver.krylov_solver
    opts = petsc4py.PETSc.Options()
    option_prefix = ksp.getOptionsPrefix()
    opts[f"{option_prefix}ksp_type"] = options0['ksp_type']
    opts[f"{option_prefix}pc_type"] = options0['pc_type']
    opts[f"{option_prefix}pc_factor_mat_solver_type"] = options0[
        'pc_factor_mat_solver_type']
    ksp.setFromOptions()
    return solver


# Post processing:


# Fix it for vectors
def errors_L(space, uS, uEx):
    """Compute error norm on boundary

    Args:
        uS (Function): Numeric solution
        uEx (Function): Exact or model solution

    Returns:
        List: L1 and L2 norms
    """
    domain = space.mesh

    L1_scalar = fem.assemble_scalar(form((uS-uEx) * dx))
    L2_scalar = fem.assemble_scalar(form((uS - uEx)**2 * dx))

    L1_err = np.abs(domain.comm.allreduce(L1_scalar, op=MPI.SUM))
    L2_err = np.sqrt(domain.comm.allreduce(L2_scalar, op=MPI.SUM))
    return (L1_err, L2_err)


def line_collision(domain, line_cord):
    """Generate points and cells of colliding domain and line

    Args:
        domain (mesh): Domain
        line_cord (array): 3D line contervertor of coordinates 

    Returns:
        Tuple: Collision points and collision cells
    """
    bb_tree = dolfinx.geometry.BoundingBoxTree(domain, domain.topology.dim)

    cells_on_line = []
    points_on_line = []
    cell_candidates = dolfinx.geometry.compute_collisions(bb_tree, line_cord.T)
    colliding_cells = dolfinx.geometry.compute_colliding_cells(
        domain, cell_candidates, line_cord.T
        )
    for i, point in enumerate(line_cord.T):
        if len(colliding_cells.links(i)) > 0:
            points_on_line.append(point)
            cells_on_line.append(colliding_cells.links(i)[0])

    points_on_line = np.array(points_on_line, dtype=np.float64)

    return (points_on_line, cells_on_line)


def graph2D(fig, lists, natural_show=False, points_on=False):
    """Create graph from fem.Function

    Args:
        fig (plt.Figure): Figure
        lists (fem.Function , plt.Axes, str): List of (u, curent axes, title)
        method (bool): Graph method True = tripcolor, False = tricontourf
    """

    def data_construct(dofs, x_array):
        data = np.column_stack((dofs[:, 0:2], x_array))
        x_data = data[:, 0]
        y_data = data[:, 1]
        z_data = data[:, 2]
        return [x_data, y_data, z_data]

    for list in lists:
        fig, ax = plt.subplots()
        plt.close()
        u, ax, title = list
        dofs = u.function_space.tabulate_dof_coordinates()
        ax.set_title(title)
        data = data_construct(dofs, u.x.array)

        if points_on: ax.plot(data[0], data[1], 'o', markersize=2, color='grey')

        if natural_show:
            plot = ax.tripcolor(*data)
        else:
            try:
                levels = np.linspace(u.x.array.min(), u.x.array.max(), 10)
                plot = ax.tricontourf(
                    *data,
                    levels=levels,
                    )
            except:
                print(f'{title} - error')

        ax.set_aspect(1)
        fig.colorbar(plot, ax=ax)
    return

In [2]:
class Problem:

    def __init__(
        self,
        a: ufl.Form,
        L: ufl.Form,
        u: fem.Function = None,
        bcs: typing.List[fem.petsc.DirichletBCMetaClass] = [],
        petsc_options={
            'ksp_type': 'preonly', 'pc_type': 'lu'
            },
        assemble_options={
            'assebmle_A': True, 'assemble_B': True
            },
        ghost_opions={
            'addv': fem.petsc.PETSc.InsertMode.ADD,
            'mode': fem.petsc.PETSc.ScatterMode.REVERSE,
            },
        form_compiler_params={},
        jit_params={},
        ):

        def set_options(self):
            problem_prefix = f'dolfinx_solve_{id(self)}'
            self._solver.setOptionsPrefix(problem_prefix)
            opts = fem.petsc.PETSc.Options()
            opts.prefixPush(problem_prefix)
            for k, v in petsc_options.items():
                opts[k] = v
            opts.prefixPop()
            self._solver.setFromOptions()
            self._A.setOptionsPrefix(problem_prefix)
            self._A.setFromOptions()
            self._b.setOptionsPrefix(problem_prefix)
            self._b.setFromOptions()

        # Creating u function
        if u is None:
            # Extract function space from TrialFunction (which is at the
            # end of the argument list as it is numbered as 1, while the
            # Test function is numbered as 0)
            self._u = fem.Function(a.arguments()[-1].ufl_function_space())
        else:
            self._u = u

        # Forms and bcs
        self._bcs = bcs

        self._a = fem.form(
            a,
            form_compiler_params=form_compiler_params,
            jit_params=jit_params,
            )
        self._A = fem.petsc.create_matrix(self._a)

        self._L = fem.form(
            L,
            form_compiler_params=form_compiler_params,
            jit_params=jit_params,
            )
        self._b = fem.petsc.create_vector(self._L)

        # Creating solver
        self._solver = fem.petsc.PETSc.KSP().create(
            self._u.function_space.mesh.comm
            )
        self._solver.setOperators(self._A)
        set_options(self)

        # Another options
        self._ghost_opions = ghost_opions
        self._assemble_options = assemble_options

    def assemble_A(self):
        self._A.zeroEntries()
        fem.petsc._assemble_matrix_mat(self._A, self._a, bcs=self._bcs)
        self._A.assemble()

    def assemble_b(self):
        with self._b.localForm() as b_loc:
            b_loc.set(0)
        fem.petsc.assemble_vector(self._b, self._L)

        fem.petsc.apply_lifting(self._b, [self._a], bcs=[self._bcs])
        self._b.ghostUpdate(
            addv=self._ghost_opions['addv'],
            mode=self._ghost_opions['mode'],
            )
        fem.petsc.set_bc(self._b, self._bcs)

    def solve(self):
        if self._assemble_options['assebmle_A'] == True: self.assemble_A()
        if self._assemble_options['assebmle_A'] == True: self.assemble_b()
        self._solver.solve(self._b, self._u.vector)
        self._u.x.scatter_forward()
        return self._u

    @classmethod
    def KSP_types():
        return fem.petsc.PETSc.KSP.Type

    @classmethod
    def PC_types():
        return fem.petsc.PETSc.PC.Type

    @classmethod
    def ghost_updates():
        return (fem.petsc.PETSc.InsertMode, fem.petsc.PETSc.ScatterMode)

    # @classmethod
    # def HYPER_types():
    # return fem.petsc.PETSc

    @property
    def L(self) -> fem.FormMetaClass:
        """The compiled linear form"""
        return self._L

    @property
    def a(self) -> fem.FormMetaClass:
        """The compiled bilinear form"""
        return self._a

    @property
    def A(self) -> fem.petsc.PETSc.Mat:
        """Matrix operator"""
        return self._A

    @property
    def b(self) -> fem.petsc.PETSc.Vec:
        """Right-hand side vector"""
        return self._b

    @property
    def solver(self) -> fem.petsc.PETSc.KSP:
        """Linear solver object"""
        return self._solver

In [3]:
L = 1
W = 0.2
domain = mesh.create_box(
    MPI.COMM_WORLD,
    [np.array([0, 0, 0]), np.array([L, W, W])],
    [50, 3, 3],
    cell_type=mesh.CellType.hexahedron,
    )
create_connectivity(domain=domain)
V = VectorFunctionSpace(domain, ('CG', 1))

u, v = TrialFunction(V), TestFunction(V)
x, y, z = SpatialCoordinate(V)
ds = ufl.Measure("ds", domain=domain)
dx = ufl.Measure('cell', subdomain_id='everywhere')

delta = 0.4 * (W / L)**2
lamb = 1.25
mu = 1
rho = 2

T = vector(0, 0, 0)
uD1 = vector(0, 0, 0)
uD2 = vector(0, 0, -0.2)
f = vector(0, 0, -rho * delta)
epsilon = lambda u: sym(grad(u))
sigma = lambda u: lamb * div(u) * I(u) + 2 * mu * epsilon(u)

bcs = [
    DirichletBC(V, Function(V, uD1), lambda x: np.isclose(x[0], 0)),
    DirichletBC(V, Function(V, uD2), lambda x: np.isclose(x[0], 1)),
    ]

F = (sigma(u)|inner|epsilon(v)) * dx - (f|dot|v) * dx - (T|dot|v) * ds

In [4]:
problem = Problem(
    lhs(F),
    rhs(F),
    bcs=bcs,
    petsc_options={
        "ksp_type": "preonly", "pc_type": "lu"
        },
    )
uS_f = problem.solve()
uS_f.name = 'Deformation'

In [9]:
s = sigma(uS_f) - 1 / 3 * tr(sigma(uS_f)) * I(uS_f)
estress = sqrt(3 / 2 * s**2)

V_von = FunctionSpace(domain, ('CG', 2))
stress_f = Function(V_von, estress)
stress_f.name = 'Stress von Mises'

V2 = VectorFunctionSpace(domain, ('CG', 2))
uS2 = Function(V2,uS_f)

In [10]:
import shutil
try:
    shutil.rmtree('/home/VTK/Elasticity_files')
except:
    print('Directory empty yet')

with io.VTKFile(
    domain.comm, '/home/VTK/Elasticity_files/elasticity.pvd', 'w'
    ) as file:
    file.write_function([uS2,stress_f])

# Problem with div grad

In [11]:
V2 = fem.TensorFunctionSpace(domain, ('CG', 1))
f1 = Function(
    V,
    ufl.nabla_div(sym(ufl.nabla_grad(uS_f)).T),
    )
print(f1.x.array[0:6], f1.x.array[:-6:-1])
f1.name = 'f1'

f21 = fem.Function(V2)
f21.interpolate(fem.Expression(ufl.grad(uS_f), V2.element.interpolation_points()))
f22 = Function(V, ufl.div(f21))
print(f22.x.array[0:6], f22.x.array[:-6:-1])
f22.name = 'f2'

f31 = fem.Function(V2)
f31.interpolate(
    fem.Expression(
        ufl.grad(uS_f) + ufl.grad(uS_f).T, V2.element.interpolation_points()
        )
    )
f32 = Function(V, ufl.div(f31))
print(f32.x.array[0:6], f32.x.array[:-6:-1])
f32.name = 'f3'

f4 = Function(
    V,
    0.5 * ufl.div((ufl.grad(uS_f) + ufl.grad(uS_f).T)),
    )
print(f4.x.array[0:6], f4.x.array[:-6:-1])
f4.name = 'f4'

[1.0206744  0.09590571 0.77288126 0.17956393 0.01613375 0.53729754] [-0.4049542  -0.06761252 -0.62327229 -0.33807528 -0.06666642]
[ 0.53003161  2.74768222  0.31271893  0.2487808   0.62629139 -1.17671347] [0. 0. 0. 0. 0.]
[ 3.10141202  2.93949364  1.85848146  1.07816048  0.44488119 -0.38363733] [-0.8099084  -0.13522503 -1.24654459 -0.67615056 -0.13333285]
[1.0206744  0.09590571 0.77288126 0.17956393 0.01613375 0.53729754] [-0.4049542  -0.06761252 -0.62327229 -0.33807528 -0.06666642]


In [12]:
with io.VTKFile(
    domain.comm, '/home/VTK/Elasticity_files/test.pvd', 'w'
    ) as file:
    file.write_function([f1, f22, f32, f4])